
# YOLOv8 Object Detection with Early Stopping (Patience) and FPS Overlay

This notebook trains a YOLOv8 model on a custom litter detection dataset, using early stopping based on patience to prevent overfitting. 
The notebook includes live training visualization and overlays FPS on the output video for real-time performance analysis.


In [3]:
import numpy
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [4]:
import os
import torchvision
import torch
from ultralytics import YOLO
import cv2
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output
os.environ["YOLOv8_DDP_DISABLE"] = "True"
# Define paths
root_path = 'C:/Users/Priyam/OneDrive/Documents/Capstone gpt/data'
model_path = os.path.join(root_path, 'yolov8l.pt')
image_path = os.path.join(root_path, 'images')
label_path = os.path.join(root_path, 'labels')
video_path = 'C:/Users/Priyam/OneDrive/Documents/Capstone gpt/data/test_litter_small.mp4'
output_video_path = 'C:/Users/Priyam/OneDrive/Documents/Capstone gpt/data/output_test_litter_small_annotated.mp4'
yaml_path = os.path.join(root_path, 'litter_dataset.yaml')

In [5]:
torch.cuda.empty_cache() 

In [6]:
yaml_content = '''
path: C:/Users/Priyam/OneDrive/Documents/Capstone gpt/data/
train: images/train
val: images/val
test: images/test
names:
  0: litter
'''

with open(yaml_path, 'w') as file:
    file.write(yaml_content)

In [7]:

# Set device to GPU (CUDA) if available, otherwise fallback to CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
model = YOLO(model_path).to(device)


Using device: cuda


In [ ]:
def train_yolo_with_early_stopping(model, yaml_path ,img_size, epochs, patience):
    best_mAP50, wait = 0, 0
    mAP50_values= []

    train_params = {
    'epochs': epochs,       # Set to the passed epochs parameter
    'lr0': 0.001,           # Initial learning rate
    'lrf': 0.2,             # Final learning rate fraction
    'optimizer': 'SGD',     # Optimizer choice
    'cos_lr': True,         # Cosine annealing scheduler   
    'batch': 16,       # Adjust batch size based on GPU memory
    'half': True,            # Use mixed precision for efficiency
    'augment': True
    }
    results = model.train(data=yaml_path, imgsz=640, **train_params, patience=10)
    mAP50 = results.box.map50
    mAP50_values.append(mAP50)

    for i in range(1, len(mAP50_values)):
        if mAP50_values[i] > best_mAP50:
            best_mAP50, wait = mAP50_values[i], 0 
        else:
            wait += 1
            if wait >= patience:
                print(f"Early stopping triggered at epoch {i + 1} with mAP50: {best_mAP50:.3f}")
                break
    return results


In [ ]:

# Train using the specified device
results_640 = train_yolo_with_early_stopping(model, yaml_path ,img_size=640, epochs=50, patience=10)


engine\trainer: task=detect, mode=train, model=C:/Users/Priyam/OneDrive/Documents/Capstone gpt/data\yolov8l.pt, data=C:/Users/Priyam/OneDrive/Documents/Capstone gpt/data\litter_dataset.yaml, epochs=50, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train60, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=True, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=

train: Scanning C:\Users\Priyam\OneDrive\Documents\Capstone gpt\data\labels\train.cache... 359 images, 1 backgrounds, 0 corrupt: 100%|██████████| 360/360 [00:00<?, ?it/s]
val: Scanning C:\Users\Priyam\OneDrive\Documents\Capstone gpt\data\labels\val.cache... 43 images, 2 backgrounds, 0 corrupt: 100%|██████████| 45/45 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 103 weight(decay=0.0), 110 weight(decay=0.0005), 109 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      6.09G      2.307      2.667      1.486        100        640: 100%|██████████| 45/45 [00:18<00:00,  2.47it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:07<00:00,  2.57s/it]

                   all         45        140    0.00319      0.307    0.00219   0.000703


...

      50/50      5.84G      1.636     0.9282       1.24         87        640: 100%|██████████| 45/45 [00:17<00:00,  2.58it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.75it/s]

                   all         45        140      0.717      0.414      0.524      0.298



50 epochs completed in 0.908 hours.
Optimizer stripped from runs\detect\train38\weights\last.pt, 79.3MB
Optimizer stripped from runs\detect\train38\weights\best.pt, 79.3MB

Validating runs\detect\train38\weights\best.pt...
Ultralytics 8.3.5  Python-3.12.4 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1080Ti Laptop GPU, 8188MiB)
Model summary (fused): 286 layers, 39,433,555 parameters, 0 gradients, 145.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.45it/s]


                   all         45        140      0.820      0.504      0.71      0.372
Speed: 0.3ms preprocess, 14.5ms inference, 0.0ms loss, 3.2ms postprocess per image


In [ ]:
def test_video_on_yolo_with_fps_overlay(model, video_path, output_path, img_size):
    cap = cv2.VideoCapture(video_path)
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 30.0, (int(cap.get(3)), int(cap.get(4))))
    frame_count, total_time = 0, 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        start_time = time.time()
        results = model(frame, imgsz=img_size)
        end_time = time.time()
        fps =( 1 / (end_time - start_time))
        total_time += (end_time - start_time)
        frame_count += 1
        #annotated_frame = results.render()[0]
        annotated_frame = results[0].plot()
        cv2.putText(annotated_frame, f"FPS: {fps:.2f}", (annotated_frame.shape[1] - 150, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        out.write(annotated_frame)
    cap.release()
    out.release()
    return frame_count/total_time

# Function to validate model and get mAP50
def validate_yolo_model(model, img_size):
    results = model.val(imgsz=img_size)  # This runs validation on the set validation dataset
    mAP50 = results.results_dict.get('mAP50', "Not Available")  # Fetches mAP50 if available
    return mAP50



In [11]:
mAP50_640 = validate_yolo_model(model, img_size=640)
fps_640 = test_video_on_yolo_with_fps_overlay(model, video_path, output_video_path.replace('.mp4', '_640.mp4'), img_size=640)

Ultralytics 8.3.5  Python-3.12.4 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1080Ti Laptop GPU, 8188MiB)
Model summary (fused): 286 layers, 39,433,555 parameters, 0 gradients, 145.2 GFLOPs


val: Scanning C:\Users\Priyam\OneDrive\Documents\Capstone gpt\data\labels\val.cache... 43 images, 2 backgrounds, 0 corrupt: 100%|██████████| 45/45 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:11<00:00,  1.98s/it]


                   all         45        140      0.705      0.496      0.581      0.317
Speed: 1.3ms preprocess, 32.9ms inference, 0.0ms loss, 3.0ms postprocess per image

0: 640x640 9 litters, 18.4ms
Speed: 6.1ms preprocess, 18.4ms inference, 2.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 11 litters, 18.6ms
Speed: 5.5ms preprocess, 18.6ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 11 litters, 26.6ms
Speed: 7.6ms preprocess, 26.6ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 11 litters, 26.8ms
Speed: 5.4ms preprocess, 26.8ms inference, 3.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 11 litters, 26.6ms
Speed: 3.6ms preprocess, 26.6ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 11 litters, 27.1ms
Speed: 4.1ms preprocess, 27.1ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 11 litters, 26.5ms
Speed: 2.5ms preprocess, 26.5m

In [12]:
print(f"FPS: {fps_640}")

FPS: 40
